In [125]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
import datetime as dt
from statistics import stdev
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from IPython.display import clear_output
import numpy as np
import random
random.seed(42)
sys_random = random.SystemRandom()
import joblib
import pickle

# Scoring Data Using Keras Regressor

### Matt Mitchell | Sumit Dhar

In [126]:
cols = ['unit', 'cycles', 'op_setting1', 'op_setting2', 'op_setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']

# Reading Test Data

In [127]:
tests=[]
for i in range(0,4):
    df= pd.read_csv('test_FD00'+str(i+1)+'.txt',  delimiter=" ", decimal=".", header=None)
    df.drop(df.columns[list(df.columns[df.isnull().sum() > 0])], axis=1, inplace=True)
    df.columns = cols
    df['unit']= df['unit'].apply(lambda x: (i*1000+x))
    tests.append(df)

In [128]:
test = pd.concat(tests, axis=0)

In [129]:
len(test.unit.unique())

707

# Scoring Function

- Read Keras Model
- Read List of features from pickle
- Read Scikit learn Scaler from a joblib file

In [130]:
def read_model_score(test, model_num=4, version=0, aggr=['min']):
    scored=[]
    for k in range(model_num):
        feature_file= 'feature_''scaler_'+str(k+1)+'_'+str(version)+'.pickle'
        with open(feature_file, 'rb') as handle:
            features = pickle.load(handle)
        scal_name= 'scaler_'+str(k+1)+'_'+str(version)+'.joblib'
        xscaler= joblib.load(scal_name)
        m_name= 'model_'+str(k+1)+'_'+str(version)
        model = keras.models.load_model(m_name)
        test_x= test.loc[:,features].values
        test_X= xscaler.transform(test_x)
        test_u= test[['unit', 'cycles']]
        ypred= model.predict(test_X).reshape((-1,))
        ypred[ypred==0]= 0.00001
        test_u.loc[:,'pMaxCycles']= test_u.loc[:,'cycles']/ypred
        max_cyles= pd.DataFrame(test_u.groupby('unit')['cycles'].agg('max'))
        max_pMaxCycles = pd.DataFrame(test_u.groupby('unit')['pMaxCycles'].agg('max'))
        test_out= max_cyles.merge(max_pMaxCycles, left_index= True, right_index= True)
        test_out['pRUL']= (test_out.iloc[:,1]- test_out.iloc[:,0]).apply(lambda x: max(x,0)).apply(lambda x: int(x))
        scored.append(test_out['pRUL'])
    combined= pd.concat(scored, axis=1)
    combined.columns= ['Fail Mode 1', 'Fail Mode 2', 'Fail Mode 3', 'Fail Mode 4']
    combined['Final Effect RUL']= combined.min(axis=1)
    return(combined)

    

In [131]:
scored= read_model_score(test,4,3)

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

# Thresholding

In [132]:
scored['Operating-Stage']= 'Normal'
scored.loc[scored['Final Effect RUL'] <=10, 'Operating-Stage']= 'Critical'
scored.loc[scored['Final Effect RUL'] < 5, 'Operating-Stage']= 'Super Critical'

In [133]:
scored['Operating-Stage'].value_counts()

Normal            645
Critical           36
Super Critical     26
Name: Operating-Stage, dtype: int64

In [134]:
scored.sort_values(by=['Final Effect RUL'], inplace= True)

In [135]:
sqdrns= ['33d Fighter Wing AETC-58th Fighter Squadron','53d Test and Evaluation Group-31st TE Squadron',\
         '56th Fighter Wing AETC-62nd Fighter Squadron','56th Fighter Wing AETC-61st Fighter Squadron',\
         '56th Fighter Wing AETC-63rd Fighter Squadron','56th Fighter Wing AETC-308th Fighter Squadron']

In [136]:
random.seed(42)
scored['squadron']= np.random.choice(sqdrns, size=len(scored))

In [137]:
scored.to_csv('failure-predictions.csv',index=True)

# Scored File

In [138]:
scored

,Fail Mode 1,Fail Mode 2,Fail Mode 3,Fail Mode 4,Final Effect RUL,Operating-Stage,squadron
unit,,,,,,,
1205,0,10,0,9,0,Super Critical,56th Fighter Wing AETC-308th Fighter Squadron
3203,0,88,18,57,0,Super Critical,53d Test and Evaluation Group-31st TE Squadron
3169,0,19,0,12,0,Super Critical,56th Fighter Wing AETC-61st Fighter Squadron
1175,6,43,0,28,0,Super Critical,56th Fighter Wing AETC-63rd Fighter Squadron
3014,0,39,0,30,0,Super Critical,33d Fighter Wing AETC-58th Fighter Squadron
...,...,...,...,...,...,...,...
2077,395,282,116,119,116,Normal,53d Test and Evaluation Group-31st TE Squadron
3129,183,263,117,145,117,Normal,33d Fighter Wing AETC-58th Fighter Squadron
3034,211,288,131,164,131,Normal,53d Test and Evaluation Group-31st TE Squadron
